In [1]:
%pip install pydantic

# pip install pydantic[email]

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Without Type hinting:

In [3]:
def get_full_name(first_name, last_name):
    full_name = first_name.title() + " " + last_name.title()
    return full_name


print(get_full_name("john", "doe"))

John Doe


With Type hinting:

In [4]:
def get_full_name(first_name: str, last_name: str) -> str:
    full_name = first_name.title() + " " + last_name.title()
    return full_name


print(get_full_name("john", "doe"))


John Doe


Simple Pydantic Model Example

In [5]:
from pydantic import BaseModel

In [6]:
class User(BaseModel):
    id: int
    name: str = 'Jane Doe'

In [7]:
user = User(id='123')

In [9]:
assert user.name == 'Jane Doe'  
assert user.id == 123  
assert isinstance(user.id, int)

Nested Models

In [11]:
class Group(BaseModel):
    id: int
    members: list[User] = []


In [12]:

group = Group(id=123)

In [13]:
group.members

[]

In [14]:
group.members.append(user)

In [15]:
group.members

[User(id=123, name='Jane Doe')]

Self-referencing models are also supported


In [17]:
class User(BaseModel):
    id: int
    name: str = 'Jane Doe'
    friends: list[User] = []

In [21]:
monica = User(id=11, name='Monica')
pheebe = User(id=12, name='Pheebe')

In [22]:
monica.friends.append(pheebe)

In [23]:
monica.friends

[User(id=12, name='Pheebe', friends=[])]

Fields

In [26]:
from uuid import uuid4

from pydantic import BaseModel, Field

class User(BaseModel):
    id: str = Field(default_factory=lambda: uuid4().hex)
    name: str = Field(default='John Doe', alias='username')

In [27]:
user = User(username='johndoe')

In [28]:
user.model_dump(by_alias=True)

{'id': 'cd78f73129e24525b41f6358be48f400', 'username': 'johndoe'}

Adding Constraints

In [29]:
class NumericalConstraints(BaseModel):
    positive: int = Field(gt=0)
    non_positive: int = Field(le=0)
    even: int = Field(multiple_of=2)

In [30]:
class StringConstraints(BaseModel):
    short: str = Field(min_length=3)
    long: str = Field(max_length=10)
    regex: str = Field(pattern=r'^\d*$')

Validators

In [33]:
from datetime import date, timedelta
from uuid import UUID, uuid4
from enum import Enum
from pydantic import BaseModel, EmailStr, Field, field_validator

In [32]:
class Department(Enum):
    HR = "HR"
    SALES = "SALES"
    IT = "IT"
    ENGINEERING = "ENGINEERING"

class Employee(BaseModel):
    employee_id: UUID = Field(default_factory=uuid4, frozen=True)
    name: str = Field(min_length=1, frozen=True)
    email: EmailStr = Field(pattern=r".+@example\.com$")
    date_of_birth: date = Field(alias="birth_date", repr=False, frozen=True)
    salary: float = Field(alias="compensation", gt=0, repr=False)
    department: Department
    elected_benefits: bool

    @field_validator("date_of_birth")
    @classmethod
    def check_valid_age(cls, date_of_birth: date) -> date:
        today = date.today()
        eighteen_years_ago = date(today.year - 18, today.month, today.day)

        if date_of_birth > eighteen_years_ago:
            raise ValueError("Employees must be at least 18 years old.")

        return date_of_birth

In [34]:
young_employee_data = {
    "name": "John Doe",
    "email": "johndoe@example.com",
    "birth_date": date.today() - timedelta(days=365 * 17),
    "compensation": 90_000,
    "department": "SALES",
    "elected_benefits": True,
}

In [36]:
Employee(**young_employee_data)

ValidationError: 1 validation error for Employee
birth_date
  Value error, Employees must be at least 18 years old. [type=value_error, input_value=datetime.date(2007, 9, 29), input_type=date]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error

In [35]:
Employee.model_validate(young_employee_data)

ValidationError: 1 validation error for Employee
birth_date
  Value error, Employees must be at least 18 years old. [type=value_error, input_value=datetime.date(2007, 9, 29), input_type=date]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error